In [ ]:
import os
locs = ['beginning', 'middle', 'end']
primary_task = ['qa', 'translate', 'summarize', 'extract', 'style', 'evaluate']
secondary_task = ['alpaca', 'translate', 'summarize', 'extract', 'style', 'evaluate', 'witness']

for p_task in primary_task:
    for s_task in secondary_task:
        if p_task == s_task: continue 
        for loc in locs:
            if p_task == 'qa' and s_task == 'alpaca': 
                number = 60000
            elif p_task == 'qa' or s_task == 'alpaca': 
                number = 8000
            else:
                number = 2000
            status = os.system("python ./prepare_training_dataset.py --orig_task " + p_task + " --emb_task "+ s_task + " --number " + str(number) + " --embed_loc "+loc)

In [ ]:
import json
huge_dataset = []
for file_ in os.listdir('dataset_training_sampled_out'):
    huge_dataset.extend(json.load(open(os.path.join('dataset_training_sampled_out',file_))))

In [ ]:
#remove duplicated of final_aggregated_prompt
#note that the combination of primary task + text may be repeated 
    #as we originally trained triplet models, we made sure the whole combination (final_aggregated_prompt) is unique 
    #check for primary task + text as well if needed 
    
    
unique_text = set()
new_huge_dataset = []
repeated = 0
for i in range(0,len(huge_dataset)):
    item = huge_dataset[i]
    if 'final_prompt' in item.keys(): 
        if item['final_prompt'] in unique_text:
            repeated += 1 
        else:
            unique_text.add(item['final_prompt'] )
            new_huge_dataset.append(item)
    
    if 'final_aggregated_prompt' in item.keys(): 
        if item['final_aggregated_prompt'] in unique_text:
            repeated += 1 
        else:
            unique_text.add(item['final_aggregated_prompt'])
            new_huge_dataset.append(item)    
huge_dataset = new_huge_dataset 

In [ ]:
from random import shuffle
shuffle(huge_dataset)

In [ ]:

# We use this for training only, the val and test are from OOD data
with open('dataset_sampled/train_subset.json', 'w') as fout:
    json.dump(huge_dataset , fout)